# Single Agent Team in Autogen

Perfect for tasks where one agent can handle everything but we want a team like-control.

In [3]:
import asyncio
from autogen_agentchat.agents import AssistantAgent

from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)



# Why use a single Agent team?

1. Consistency - termination conditions, logging, observing
2. Scalability - we can add more agents as our use case expands without rewriting logic. 
3. Control - Features like observers or cancellation work seamlessly.

In [5]:
poet = AssistantAgent(
    name = 'solo_poet',
    model_client=model_client,
    system_message="You are a poet. Write a short poem on Moon and end with 'MOONLIGHT FADES'. "
)

termination = TextMentionTermination('MOONLIGHT FADES')

team = RoundRobinGroupChat(
    participants=[poet],
    termination_condition=termination
)

In [7]:
async def test_single_agent():
    task = TextMessage(content="Write a short poem on Moon",source='user')
    result = await team.run(task = task)
    for message in result.messages:
        print(f"{message.source}: {message.content}")


await test_single_agent() 

user: Write a short poem on Moon
solo_poet: In the velvet sky she gently sails,  
Casting silver across the night's black veils,  
Whispers of secrets the stars confide,  
In her radiant glow, our dreams reside.  

Through ancient tales and lovers' sighs,  
Her light, a beacon in darkened skies,  
A guardian of the night's embrace,  
With quiet grace, she's slipped into place.  

But as the dawn's first colors invade,  
Her tender glow begins to evade,  
And with a soft, ethereal glide,  
She disappears as MOONLIGHT FADES.  


## Single-Agent Team vs. Plain Single Agent

| Aspect             | Single-Agent Team            | Plain Single Agent          |
|--------------------|------------------------------|-----------------------------|
| **Termination**    | Stops with condition (e.g., "MOONLIGHT FADES") | Runs until task ends        |
| **Observability**  | Supports Console observers   | Manual logging only         |
| **Flexibility**    | Easily scales to multi-agent | Fixed to one agent          |